In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!


#from huggingface_hub import notebook_login

#notebook_login()

In [2]:
import json
import pandas as pd
from copy import deepcopy
import random
from tqdm.notebook import tqdm

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AdamW
)
import pickle

from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [3]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [4]:
def split_input_dict(input_dict, ratio = 0.01, seed = 42):
    split_point = int(len(input_dict['utterance']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['utterance'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['utterance'] = input_dict['utterance'][:split_point]
    train_dict['utterance'] = input_dict['utterance'][split_point:]
    return train_dict, valid_dict

In [5]:
def read_input(path):
    data = pd.read_csv(path)
    input_dict = {"utterance":[]}
    for _, row in tqdm(data.iterrows()):
        for q_col in ['질문_1', '질문_2']:
            for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
                input_dict["utterance"].append({"question":row[q_col],"answer":row[a_col]})
            
            
    train_dict,valid_dict = split_input_dict(input_dict)
    train_question = []
    train_answer = []
    for pairs in tqdm(train_dict['utterance']):
        train_question.append(pairs['question'])
        train_answer.append(pairs['answer'])

    valid_question = []
    valid_answer = []
    for pairs in tqdm(valid_dict['utterance']):
        valid_question.append(pairs['question'])
        valid_answer.append(pairs['answer'])

    
    return train_question ,train_answer , valid_question , valid_answer

In [6]:
train_q ,train_a , valid_q , valid_a = read_input("./bigdata/train.csv")

0it [00:00, ?it/s]

  0%|          | 0/6376 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [7]:
####데이터 3개씩 끊어보는 기능
n=2
for ind,i in enumerate(zip(train_q,train_a)):
    if ind>3:
        break
    print(i[0]+" ## "+i[1])

탄화목의 기대수명은 얼마야? ## 탄화목의 기대수명은 탄화목의 등급에 있어서 WTDCA(WorldTimber Deck & Cladding Asssociation)룰에 따르면, Class D등급은 최소 25년, Class C등급은 15년의 내구성을 보장합니다.
차음재의 다양한 종류에는 어떤 것들이 있는지 알고 싶습니다. ## 차음재에는 주로 단일재와 이중벽이 사용됩니다. 또한 플라스틱, 콘크리트, 목재, 알루미늄, 유리섬유 등 다양한 재료가 사용되며, 사용하는 재료에 따라 소음 절연 효과에도 차이가 있을 수 있습니다.
그라스울의 장점이 뭐야? ## 그라스울은 단열성능이 뛰어나고 운반, 보관 및 취급이 용이하다는 것이 가장 큰 장점입니다. 또한 그라스울은 비산화규소 섬유로서 내화성, 내부 마찰율, 내결합율이 높고 절연성이 우수하여 고온에서 사용할 때 안정성이 뛰어나다는 장점도 있습니다.
중목구조 방식이 뭐야? ## 중목구조는 건물을 지지하기 위해 중앙에 큰 기둥(중목)을 설치하고 그 주변에 보(보근)를 배열하는 구조 방식입니다. 이 방식은 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다. 중목구조는 구조물에 안정성을 제공하며, 전통적으로 동양 건축에서 사용된 구조 방식 중 하나입니다.


In [8]:
class PairedDataset:
    def __init__(self, src_tokenizer, tgt_tokenizer,file_question , file_answer):
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = tgt_tokenizer
        self.data_q = file_question
        self.data_a = file_answer
        self.data = [[self.data_q[i],self.data_a[i]] for i in range(len(self.data_q)) ]
    def __getitem__(self, index):
        src, trg = self.data_q[index] , self.data_a[index]
        embeddings = self.src_tokenizer(src, return_attention_mask=False, return_token_type_ids=False)
        embeddings['labels'] = self.trg_tokenizer(trg, return_attention_mask=False)['input_ids']

        return embeddings

    def __len__(self):
        return len(self.data)

In [9]:
dataset = PairedDataset(src_tokenizer, trg_tokenizer, train_q , train_a)
eval_dataset = PairedDataset(src_tokenizer, trg_tokenizer, valid_q , valid_a)

In [10]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    'monologg/kobert',
    'skt/kogpt2-base-v2',
    pad_token_id=trg_tokenizer.bos_token_id
)





model.config.decoder_start_token_id = trg_tokenizer.bos_token_id
model

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.6.crossattention.q_attn.bias', 'transformer.h.9.ln_cross_attn.bias', 'transformer.h.11.ln_cross_attn.weight', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.11.crossattention.q_attn.weight', 'transformer.h.2.crossattention.q_attn.bias', 'transformer.h.7.crossattention.q_attn.bias', 'transformer.h.10.crossattention.c_proj.weight', 'transformer.h.11.crossattention.c_attn.weight', 'transformer.h.4.crossattention.c_attn.bias', 'transformer.h.10.ln_cross_attn.bias', 'transformer.h.4.ln_cross_attn.bias', 'transformer.h.2.ln_cross_attn.weight', 'transformer.h.9.crossattention.q_attn.bias', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.7.ln_cross_attn.weight', 'transformer.h.6.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.3.crossattention.q_attn.bias', 'transformer.h.6.crossat

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [11]:
collator = DataCollatorForSeq2Seq(src_tokenizer, model)

arguments = Seq2SeqTrainingArguments(
    output_dir='dacon-v0',
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    num_train_epochs=80,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,
    save_total_limit=5,
    dataloader_num_workers=1,
    fp16=True,
    load_best_model_at_end=True,
    #push_to_hub=True,
    learning_rate = 5e-5
)

trainer = Seq2SeqTrainer(
    model,
    arguments,
    data_collator=collator,
    train_dataset=dataset,
    eval_dataset=eval_dataset
)

In [12]:
trainer.train()

model.save_pretrained("checkpoints/dacon-v0/best_model")

/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss


RuntimeError: CUDA out of memory. Tried to allocate 1.75 GiB (GPU 0; 23.69 GiB total capacity; 21.55 GiB already allocated; 11.19 MiB free; 22.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
#trainer.push_to_hub()